# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing Turi Create functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in Turi Create functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up Turi Create

In [17]:
import pandas as pd
from sklearn import linear_model

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [18]:
sales=pd.read_csv("kc_house_data.csv")
train_data=pd.read_csv("kc_house_train_data.csv")
test_data=pd.read_csv("kc_house_test_data.csv")

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let Turi Create pick a random seed for you).  

In [19]:
#train_data,test_data = sales.random_split(.8,seed=0)


# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [5]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
#example_model = turicreate.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                    #validation_set = None)
reg=linear_model.LinearRegression()
reg.fit(train_data[example_features],train_data[['price']])

LinearRegression()

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [6]:
example_weight_summary =reg.coef_
print (example_weight_summary)

[[   315.40669062 -65081.88711588   6942.16598637]]


# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing Turi Create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [20]:
example_predictions = reg.predict(train_data[example_features])
print (example_predictions[0]) # should be 271789.505878

[271789.26537997]


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [61]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    price_pred= model.predict(data)
   # print("price",price_pred) 
        
    # Then compute the residuals/errors
    Diff=outcome-price_pred
    #print("Diff",Diff)
    Dif_sq=Diff*Diff
    #print("sDiff",Dif_sq)
    # Then square and add them up
    RSS=Dif_sq.sum()
    #print("rss",RSS)
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [62]:
rss_example_train = get_residual_sum_of_squares(reg, test_data[example_features], test_data[['price']])
#print (rss_example_train) # should be 2.7376153833e+14

# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [63]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [64]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [75]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['bed_bath_rooms']=train_data['bedrooms']*train_data['bathrooms']
test_data['bed_bath_rooms']=test_data['bedrooms']*test_data['bathrooms']
train_data['log_sqft_living']=train_data['sqft_living'].apply(lambda x:log(x))
test_data['log_sqft_living']=test_data['sqft_living'].apply(lambda x:log(x))
train_data['lat_plus_long']=train_data['lat']+train_data['long']
test_data['lat_plus_long']=test_data['lat']+test_data['long']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [76]:
print(test_data['bed_bath_rooms'].mean())
print(test_data['log_sqft_living'].mean())
print(test_data['bedrooms_squared'].mean())
print(test_data['lat_plus_long'].mean())

7.5039016315913925
7.550274679645921
12.4466777015843
-74.65333355403185


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [67]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using turicreate.linear_regression.create() and look at the value of the weights/coefficients:

In [77]:
# Learn the three models: (don't forget to set validation_set = None)
n_m1=linear_model.LinearRegression()
n_m1.fit(train_data[model_1_features],train_data[['price']])
n_m2=linear_model.LinearRegression()
n_m2.fit(train_data[model_2_features],train_data[['price']])
n_m3=linear_model.LinearRegression()
n_m3.fit(train_data[model_3_features],train_data[['price']])


LinearRegression()

In [78]:
# Examine/extract each model's coefficients:
print(n_m1.coef_)
print(n_m2.coef_)

[[ 3.12258646e+02 -5.95865332e+04  1.57067421e+04  6.58619264e+05
  -3.09374351e+05]]
[[ 3.06610053e+02 -1.13446368e+05 -7.14613083e+04  6.54844630e+05
  -2.94298969e+05  2.55796520e+04]]


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [79]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
print(get_residual_sum_of_squares(n_m1, train_data[model_1_features], train_data[['price']]))
print(get_residual_sum_of_squares(n_m2, train_data[model_2_features], train_data[['price']]))
print(get_residual_sum_of_squares(n_m3, train_data[model_3_features], train_data[['price']]))

price    9.678800e+14
dtype: float64
price    9.584196e+14
dtype: float64
price    9.034365e+14
dtype: float64


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [81]:
# Compute the RSS on TESTING data for each of the three models and record the values:

print(get_residual_sum_of_squares(n_m1, test_data[model_1_features], test_data[['price']]))
print(get_residual_sum_of_squares(n_m2, test_data[model_2_features], test_data[['price']]))
print(get_residual_sum_of_squares(n_m3, test_data[model_3_features], test_data[['price']]))

price    2.255005e+14
dtype: float64
price    2.233775e+14
dtype: float64
price    2.592363e+14
dtype: float64


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.